<a href="https://colab.research.google.com/github/aml7hawaiiedu/CCAPLandCoverProject/blob/main/CCAP_UNET_processing_Fall2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import shutil
import hashlib

In [3]:
# # OR Unzipping to a specific directory
!mkdir /content/CCAP

In [22]:
# !mkdir '/content/Hawaii_2005_003'
# !mkdir '/content/Hawaii_2005_003/Images'
# !mkdir '/content/Hawaii_2005_003/Labels'

In [24]:
# !mkdir '/content/Hawaii_2005_004'
# !mkdir '/content/Hawaii_2005_004/Images'
# !mkdir '/content/Hawaii_2005_004/Labels'

In [71]:
!mkdir '/content/Hawaii_2005_005'
!mkdir '/content/Hawaii_2005_005/Images'
!mkdir '/content/Hawaii_2005_005/Labels'

mkdir: cannot create directory ‘/content/Hawaii_2005_005’: File exists
mkdir: cannot create directory ‘/content/Hawaii_2005_005/Images’: File exists
mkdir: cannot create directory ‘/content/Hawaii_2005_005/Labels’: File exists


In [72]:
!mkdir '/content/Hawaii_2005_006'
!mkdir '/content/Hawaii_2005_006/Images'
!mkdir '/content/Hawaii_2005_006/Labels'

mkdir: cannot create directory ‘/content/Hawaii_2005_006’: File exists
mkdir: cannot create directory ‘/content/Hawaii_2005_006/Images’: File exists
mkdir: cannot create directory ‘/content/Hawaii_2005_006/Labels’: File exists


In [27]:
!mkdir '/content/Hawaii_2005_007'
!mkdir '/content/Hawaii_2005_007/Images'
!mkdir '/content/Hawaii_2005_007/Labels'

In [28]:
!mkdir '/content/Hawaii_2005_008'
!mkdir '/content/Hawaii_2005_008/Images'
!mkdir '/content/Hawaii_2005_008/Labels'

In [4]:
!mkdir '/content/Hawaii_2005_009'
!mkdir '/content/Hawaii_2005_009/Images'
!mkdir '/content/Hawaii_2005_009/Labels'

In [ ]:
# Unzip the file and place the files to the newly cerated directory (CCAP/image_chips/ISLAND)
!unzip /content/drive/MyDrive/wetland_unet/UNET_Image_Chips/Hawaii_Island_Update/Export/Hawaii_2005_007.zip -d /content/CCAP/

Next two blocks of code are probably unnecessary but run them for a sanity check.

In [105]:
# original images and labels folder: how many files are there?
org_img = '/content/CCAP/content/Hawaii_2005_007/Images'
org_img_files = os.listdir(org_img)
org_img_count = len(org_img_files)
print(f'The number of files in the folder is: {org_img_count}')
org_label = '/content/CCAP/content/Hawaii_2005_007/Labels'
org_label_files = os.listdir(org_label)
org_label_count = len(org_label_files)
print(f'The number of files in the folder is: {org_label_count}')

The number of files in the folder is: 1476
The number of files in the folder is: 1476


In [106]:
df = pd.read_csv('/content/CCAP/content/Hawaii_2005_007/Hawaii_2005_007.csv')

In [107]:
df

,x_end,y_end,x_org,y_org,id,xmin,xmax,ymin,ymax,label,percent
0,513,513,1,1,1,266696.351583,267925.140982,2.242729e+06,2.243958e+06,0,1.0
1,1025,513,513,1,2,267925.172370,269153.967913,2.242729e+06,2.243958e+06,0,1.0
2,1537,513,1025,1,3,269153.974080,270382.751199,2.242729e+06,2.243958e+06,0,1.0
3,2049,513,1537,1,4,270382.762718,271611.577803,2.242729e+06,2.243958e+06,0,1.0
4,2561,513,2049,1,5,271611.552927,272840.338497,2.242729e+06,2.243958e+06,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
2232,16385,20982,15873,20470,1472,304789.125692,306017.925026,2.193604e+06,2.194833e+06,0,1.0
2233,16897,20982,16385,20470,1473,306017.921196,307246.719119,2.193604e+06,2.194832e+06,0,1.0
2234,17409,20982,16897,20470,1474,307246.773594,308475.540985,2.193604e+06,2.194832e+06,0,1.0
2235,17921,20982,17409,20470,1475,308475.558930,309704.336987,2.193604e+06,2.194833e+06,0,1.0


In [108]:
subset_df = df[(df['label'] > 12) & (df['label'] < 19)]

In [109]:
subset_df

,x_end,y_end,x_org,y_org,id,xmin,xmax,ymin,ymax,label,percent
1594,513,18433,1,17921,1261,266696.349041,267925.137872,2.199721e+06,2.200950e+06,13,0.064178
1595,513,18433,1,17921,1261,266696.349041,267925.137872,2.199721e+06,2.200950e+06,14,0.014835
1596,513,18433,1,17921,1261,266696.349041,267925.137872,2.199721e+06,2.200950e+06,15,0.008255
1599,1025,18433,513,17921,1262,267925.119373,269153.930565,2.199721e+06,2.200950e+06,13,0.344536
1600,1025,18433,513,17921,1262,267925.119373,269153.930565,2.199721e+06,2.200950e+06,14,0.059734
...,...,...,...,...,...,...,...,...,...,...,...
2149,2561,20982,2049,20470,1445,271611.549605,272840.339361,2.193604e+06,2.194833e+06,14,0.015591
2150,2561,20982,2049,20470,1445,271611.549605,272840.339361,2.193604e+06,2.194833e+06,15,0.042984
2154,3073,20982,2561,20470,1446,272840.328328,274069.129848,2.193604e+06,2.194833e+06,13,0.079334
2155,3073,20982,2561,20470,1446,272840.328328,274069.129848,2.193604e+06,2.194833e+06,14,0.023190


In [110]:
subset_df = subset_df.copy()  # Create a copy of the subset DataFrame to avoid modifying the original
subset_df['tif_name'] = subset_df['x_org'].astype(str) + "_" + subset_df['y_org'].astype(str) + ".tif"

In [111]:
subset_df

,x_end,y_end,x_org,y_org,id,xmin,xmax,ymin,ymax,label,percent,tif_name
1594,513,18433,1,17921,1261,266696.349041,267925.137872,2.199721e+06,2.200950e+06,13,0.064178,1_17921.tif
1595,513,18433,1,17921,1261,266696.349041,267925.137872,2.199721e+06,2.200950e+06,14,0.014835,1_17921.tif
1596,513,18433,1,17921,1261,266696.349041,267925.137872,2.199721e+06,2.200950e+06,15,0.008255,1_17921.tif
1599,1025,18433,513,17921,1262,267925.119373,269153.930565,2.199721e+06,2.200950e+06,13,0.344536,513_17921.tif
1600,1025,18433,513,17921,1262,267925.119373,269153.930565,2.199721e+06,2.200950e+06,14,0.059734,513_17921.tif
...,...,...,...,...,...,...,...,...,...,...,...,...
2149,2561,20982,2049,20470,1445,271611.549605,272840.339361,2.193604e+06,2.194833e+06,14,0.015591,2049_20470.tif
2150,2561,20982,2049,20470,1445,271611.549605,272840.339361,2.193604e+06,2.194833e+06,15,0.042984,2049_20470.tif
2154,3073,20982,2561,20470,1446,272840.328328,274069.129848,2.193604e+06,2.194833e+06,13,0.079334,2561_20470.tif
2155,3073,20982,2561,20470,1446,272840.328328,274069.129848,2.193604e+06,2.194833e+06,14,0.023190,2561_20470.tif


In [112]:
# Copy the useable images files to new folder
source_images = '/content/CCAP/content/Hawaii_2005_007/Images'
destination_images = '/content/Hawaii_2005_007/Images'
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_images, tif_name)
    destination_path = os.path.join(destination_images, tif_name)
    try:
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_images}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_images}")

Copied 1_17921.tif to /content/Hawaii_2005_007/Images
Copied 1_17921.tif to /content/Hawaii_2005_007/Images
Copied 1_17921.tif to /content/Hawaii_2005_007/Images
Copied 513_17921.tif to /content/Hawaii_2005_007/Images
Copied 513_17921.tif to /content/Hawaii_2005_007/Images
Copied 513_17921.tif to /content/Hawaii_2005_007/Images
Copied 1025_17921.tif to /content/Hawaii_2005_007/Images
Copied 1025_17921.tif to /content/Hawaii_2005_007/Images
Copied 1025_17921.tif to /content/Hawaii_2005_007/Images
Copied 1537_17921.tif to /content/Hawaii_2005_007/Images
Copied 1537_17921.tif to /content/Hawaii_2005_007/Images
Copied 1_18433.tif to /content/Hawaii_2005_007/Images
Copied 1_18433.tif to /content/Hawaii_2005_007/Images
Copied 1_18433.tif to /content/Hawaii_2005_007/Images
Copied 513_18433.tif to /content/Hawaii_2005_007/Images
Copied 513_18433.tif to /content/Hawaii_2005_007/Images
Copied 513_18433.tif to /content/Hawaii_2005_007/Images
Copied 1025_18433.tif to /content/Hawaii_2005_007/Image

In [113]:
# Copy the useable labels files to new folder
source_labels = '/content/CCAP/content/Hawaii_2005_007/Labels'
destination_labels = '/content/Hawaii_2005_007/Labels'
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_labels, tif_name)
    destination_path = os.path.join(destination_labels, tif_name)
    try:
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_labels}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_labels}")

Copied 1_17921.tif to /content/Hawaii_2005_007/Labels
Copied 1_17921.tif to /content/Hawaii_2005_007/Labels
Copied 1_17921.tif to /content/Hawaii_2005_007/Labels
Copied 513_17921.tif to /content/Hawaii_2005_007/Labels
Copied 513_17921.tif to /content/Hawaii_2005_007/Labels
Copied 513_17921.tif to /content/Hawaii_2005_007/Labels
Copied 1025_17921.tif to /content/Hawaii_2005_007/Labels
Copied 1025_17921.tif to /content/Hawaii_2005_007/Labels
Copied 1025_17921.tif to /content/Hawaii_2005_007/Labels
Copied 1537_17921.tif to /content/Hawaii_2005_007/Labels
Copied 1537_17921.tif to /content/Hawaii_2005_007/Labels
Copied 1_18433.tif to /content/Hawaii_2005_007/Labels
Copied 1_18433.tif to /content/Hawaii_2005_007/Labels
Copied 1_18433.tif to /content/Hawaii_2005_007/Labels
Copied 513_18433.tif to /content/Hawaii_2005_007/Labels
Copied 513_18433.tif to /content/Hawaii_2005_007/Labels
Copied 513_18433.tif to /content/Hawaii_2005_007/Labels
Copied 1025_18433.tif to /content/Hawaii_2005_007/Label

In [114]:
# NEW images and labels folder: how many files are there?
new_img = '/content/Hawaii_2005_007/Images'
new_img_files = os.listdir(new_img)
new_img_count = len(new_img_files)
print(f'The number of files in the folder is: {new_img_count}')
new_label = '/content/Hawaii_2005_007/Labels'
new_label_files = os.listdir(new_label)
new_label_count = len(new_label_files)
print(f'The number of files in the folder is: {new_label_count}')

The number of files in the folder is: 30
The number of files in the folder is: 30


In [115]:
# Define the file path where you want to save the CSV file
csv_file_path = '/content/Hawaii_2005_007_subset.csv'
subset_df.to_csv(csv_file_path, index=False)
print(f"DataFrame exported to {csv_file_path}")

DataFrame exported to /content/Hawaii_2005_007_subset.csv


In [116]:
# Define the path to the 'wetland_categories' folder
folder_to_zip = '/content/Hawaii_2005_007'
zip_file_path = '/content/Hawaii_2005_007_subset.zip'
shutil.make_archive(zip_file_path[:-4], 'zip', folder_to_zip)
print(f'ZIP file created: {zip_file_path}')

ZIP file created: /content/Hawaii_2005_007_subset.zip


In [117]:
# Define the destination folder in Google Drive
drive_folder = '/content/drive/MyDrive/wetland_unet/UNET_Image_Chips/imagechip_trainingdata'
shutil.move(zip_file_path, drive_folder)
print(f'ZIP file moved to Google Drive: {drive_folder}')
shutil.move(csv_file_path, drive_folder)
print(f'CSV file moved to Google Drive: {drive_folder}')

ZIP file moved to Google Drive: /content/drive/MyDrive/wetland_unet/UNET_Image_Chips/imagechip_trainingdata
CSV file moved to Google Drive: /content/drive/MyDrive/wetland_unet/UNET_Image_Chips/imagechip_trainingdata


# Define next folder

In [118]:
# Unzip the file and place the files to the newly cerated directory (CCAP/image_chips/ISLAND)
!unzip /content/drive/MyDrive/wetland_unet/UNET_Image_Chips/Hawaii_Island_Update/Export/Hawaii_2005_008.zip -d /content/CCAP/

Archive:  /content/drive/MyDrive/wetland_unet/UNET_Image_Chips/Hawaii_Island_Update/Export/Hawaii_2005_008.zip
   creating: /content/CCAP/content/Hawaii_2005_008/Images/
  inflating: /content/CCAP/content/Hawaii_2005_008/Images/1_1.tif  
  inflating: /content/CCAP/content/Hawaii_2005_008/Images/1_10241.tif  
  inflating: /content/CCAP/content/Hawaii_2005_008/Images/1_1025.tif  
  inflating: /content/CCAP/content/Hawaii_2005_008/Images/1_10753.tif  
  inflating: /content/CCAP/content/Hawaii_2005_008/Images/1_11265.tif  
  inflating: /content/CCAP/content/Hawaii_2005_008/Images/1_11777.tif  
  inflating: /content/CCAP/content/Hawaii_2005_008/Images/1_12289.tif  
  inflating: /content/CCAP/content/Hawaii_2005_008/Images/1_12801.tif  
  inflating: /content/CCAP/content/Hawaii_2005_008/Images/1_13313.tif  
  inflating: /content/CCAP/content/Hawaii_2005_008/Images/1_13825.tif  
  inflating: /content/CCAP/content/Hawaii_2005_008/Images/1_14337.tif  
  inflating: /content/CCAP/content/Hawaii_2

In [119]:
# original images and labels folder: how many files are there?
org_img = '/content/CCAP/content/Hawaii_2005_008/Images'
org_img_files = os.listdir(org_img)
org_img_count = len(org_img_files)
print(f'The number of files in the folder is: {org_img_count}')
org_label = '/content/CCAP/content/Hawaii_2005_008/Labels'
org_label_files = os.listdir(org_label)
org_label_count = len(org_label_files)
print(f'The number of files in the folder is: {org_label_count}')

The number of files in the folder is: 1476
The number of files in the folder is: 1476


In [120]:
df = pd.read_csv('/content/CCAP/content/Hawaii_2005_008/Hawaii_2005_008.csv')

In [121]:
df

,x_end,y_end,x_org,y_org,id,xmin,xmax,ymin,ymax,label,percent
0,513,513,1,1,1,266696.349253,267925.153036,2.192373e+06,2.193601e+06,10,0.040760
1,513,513,1,1,1,266696.349253,267925.153036,2.192373e+06,2.193601e+06,13,0.565292
2,513,513,1,1,1,266696.349253,267925.153036,2.192373e+06,2.193601e+06,14,0.378246
3,513,513,1,1,1,266696.349253,267925.153036,2.192373e+06,2.193601e+06,15,0.014881
4,513,513,1,1,1,266696.349253,267925.153036,2.192373e+06,2.193601e+06,21,0.000820
...,...,...,...,...,...,...,...,...,...,...,...
5934,16385,20982,15873,20470,1472,304789.146854,306017.937229,2.143247e+06,2.144476e+06,0,1.000000
5935,16897,20982,16385,20470,1473,306017.906862,307246.710622,2.143247e+06,2.144476e+06,0,1.000000
5936,17409,20982,16897,20470,1474,307246.786944,308475.556486,2.143247e+06,2.144476e+06,0,1.000000
5937,17921,20982,17409,20470,1475,308475.521470,309704.303091,2.143247e+06,2.144476e+06,0,1.000000


In [122]:
subset_df = df[(df['label'] > 12) & (df['label'] < 19)]

In [123]:
subset_df = subset_df.copy()  # Create a copy of the subset DataFrame to avoid modifying the original
subset_df['tif_name'] = subset_df['x_org'].astype(str) + "_" + subset_df['y_org'].astype(str) + ".tif"

In [124]:
subset_df

,x_end,y_end,x_org,y_org,id,xmin,xmax,ymin,ymax,label,percent,tif_name
1,513,513,1,1,1,266696.349253,267925.153036,2.192373e+06,2.193601e+06,13,0.565292,1_1.tif
2,513,513,1,1,1,266696.349253,267925.153036,2.192373e+06,2.193601e+06,14,0.378246,1_1.tif
3,513,513,1,1,1,266696.349253,267925.153036,2.192373e+06,2.193601e+06,15,0.014881,1_1.tif
6,1025,513,513,1,2,267925.107689,269153.923863,2.192373e+06,2.193601e+06,13,0.387817,513_1.tif
7,1025,513,513,1,2,267925.107689,269153.923863,2.192373e+06,2.193601e+06,14,0.126968,513_1.tif
...,...,...,...,...,...,...,...,...,...,...,...,...
4445,3585,15873,3073,15361,1087,274069.134657,275297.924941,2.155509e+06,2.156737e+06,15,0.000507,3073_15361.tif
4791,2049,16897,1537,16385,1156,270382.716590,271611.509194,2.153051e+06,2.154280e+06,15,0.000847,1537_16385.tif
4949,17409,16897,16897,16385,1186,307246.716513,308475.514444,2.153051e+06,2.154280e+06,18,0.000889,16897_16385.tif
5121,16897,17409,16385,16897,1221,306017.958385,307246.752157,2.151822e+06,2.153051e+06,16,0.002216,16385_16897.tif


In [ ]:
# Copy the useable images files to new folder
source_images = '/content/CCAP/content/Hawaii_2005_008/Images'
destination_images = '/content/Hawaii_2005_008/Images'
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_images, tif_name)
    destination_path = os.path.join(destination_images, tif_name)
    try:
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_images}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_images}")

In [ ]:
# Copy the useable labels files to new folder
source_labels = '/content/CCAP/content/Hawaii_2005_008/Labels'
destination_labels = '/content/Hawaii_2005_008/Labels'
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_labels, tif_name)
    destination_path = os.path.join(destination_labels, tif_name)
    try:
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_labels}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_labels}")

In [127]:
# NEW images and labels folder: how many files are there?
new_img = '/content/Hawaii_2005_008/Images'
new_img_files = os.listdir(new_img)
new_img_count = len(new_img_files)
print(f'The number of files in the folder is: {new_img_count}')
new_label = '/content/Hawaii_2005_008/Labels'
new_label_files = os.listdir(new_label)
new_label_count = len(new_label_files)
print(f'The number of files in the folder is: {new_label_count}')

The number of files in the folder is: 72
The number of files in the folder is: 72


In [128]:
# Define the file path where you want to save the CSV file
csv_file_path = '/content/Hawaii_2005_008_subset.csv'
subset_df.to_csv(csv_file_path, index=False)
print(f"DataFrame exported to {csv_file_path}")

DataFrame exported to /content/Hawaii_2005_008_subset.csv


In [129]:
# Define the path to the 'wetland_categories' folder
folder_to_zip = '/content/Hawaii_2005_008'
zip_file_path = '/content/Hawaii_2005_008_subset.zip'
shutil.make_archive(zip_file_path[:-4], 'zip', folder_to_zip)
print(f'ZIP file created: {zip_file_path}')

ZIP file created: /content/Hawaii_2005_008_subset.zip


In [130]:
# Define the destination folder in Google Drive
drive_folder = '/content/drive/MyDrive/wetland_unet/UNET_Image_Chips/imagechip_trainingdata'
shutil.move(zip_file_path, drive_folder)
print(f'ZIP file moved to Google Drive: {drive_folder}')
shutil.move(csv_file_path, drive_folder)
print(f'CSV file moved to Google Drive: {drive_folder}')

ZIP file moved to Google Drive: /content/drive/MyDrive/wetland_unet/UNET_Image_Chips/imagechip_trainingdata
CSV file moved to Google Drive: /content/drive/MyDrive/wetland_unet/UNET_Image_Chips/imagechip_trainingdata


# one more to do 7, 8, 9 together

In [5]:
# Unzip the file and place the files to the newly cerated directory (CCAP/image_chips/ISLAND)
!unzip /content/drive/MyDrive/wetland_unet/UNET_Image_Chips/Hawaii_Island_Update/Export/Hawaii_2005_009.zip -d /content/CCAP/

Archive:  /content/drive/MyDrive/wetland_unet/UNET_Image_Chips/Hawaii_Island_Update/Export/Hawaii_2005_009.zip
   creating: /content/CCAP/content/Hawaii_2005_009/Images/
  inflating: /content/CCAP/content/Hawaii_2005_009/Images/1_1.tif  
  inflating: /content/CCAP/content/Hawaii_2005_009/Images/1_10241.tif  
  inflating: /content/CCAP/content/Hawaii_2005_009/Images/1_1025.tif  
  inflating: /content/CCAP/content/Hawaii_2005_009/Images/1_10753.tif  
  inflating: /content/CCAP/content/Hawaii_2005_009/Images/1_11265.tif  
  inflating: /content/CCAP/content/Hawaii_2005_009/Images/1_11777.tif  
  inflating: /content/CCAP/content/Hawaii_2005_009/Images/1_12289.tif  
  inflating: /content/CCAP/content/Hawaii_2005_009/Images/1_12801.tif  
  inflating: /content/CCAP/content/Hawaii_2005_009/Images/1_13313.tif  
  inflating: /content/CCAP/content/Hawaii_2005_009/Images/1_13825.tif  
  inflating: /content/CCAP/content/Hawaii_2005_009/Images/1_14337.tif  
  inflating: /content/CCAP/content/Hawaii_2

In [6]:
# original images and labels folder: how many files are there?
org_img = '/content/CCAP/content/Hawaii_2005_009/Images'
org_img_files = os.listdir(org_img)
org_img_count = len(org_img_files)
print(f'The number of files in the folder is: {org_img_count}')
org_label = '/content/CCAP/content/Hawaii_2005_009/Labels'
org_label_files = os.listdir(org_label)
org_label_count = len(org_label_files)
print(f'The number of files in the folder is: {org_label_count}')

The number of files in the folder is: 1476
The number of files in the folder is: 1476


In [7]:
df = pd.read_csv('/content/CCAP/content/Hawaii_2005_009/Hawaii_2005_009.csv')

In [8]:
subset_df = df[(df['label'] > 12) & (df['label'] < 19)]

In [9]:
subset_df = subset_df.copy()  # Create a copy of the subset DataFrame to avoid modifying the original
subset_df['tif_name'] = subset_df['x_org'].astype(str) + "_" + subset_df['y_org'].astype(str) + ".tif"

In [10]:
subset_df

,x_end,y_end,x_org,y_org,id,xmin,xmax,ymin,ymax,label,percent,tif_name


In [11]:
# Copy the useable images files to new folder
source_images = '/content/CCAP/content/Hawaii_2005_009/Images'
destination_images = '/content/Hawaii_2005_009/Images'
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_images, tif_name)
    destination_path = os.path.join(destination_images, tif_name)
    try:
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_images}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_images}")

In [12]:
# Copy the useable labels files to new folder
source_labels = '/content/CCAP/content/Hawaii_2005_009/Labels'
destination_labels = '/content/Hawaii_2005_009/Labels'
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_labels, tif_name)
    destination_path = os.path.join(destination_labels, tif_name)
    try:
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_labels}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_labels}")

In [13]:
# NEW images and labels folder: how many files are there?
new_img = '/content/Hawaii_2005_009/Images'
new_img_files = os.listdir(new_img)
new_img_count = len(new_img_files)
print(f'The number of files in the folder is: {new_img_count}')
new_label = '/content/Hawaii_2005_009/Labels'
new_label_files = os.listdir(new_label)
new_label_count = len(new_label_files)
print(f'The number of files in the folder is: {new_label_count}')

The number of files in the folder is: 0
The number of files in the folder is: 0


In [14]:
# Define the file path where you want to save the CSV file
csv_file_path = '/content/Hawaii_2005_009_subset.csv'
subset_df.to_csv(csv_file_path, index=False)
print(f"DataFrame exported to {csv_file_path}")

DataFrame exported to /content/Hawaii_2005_009_subset.csv


In [15]:
# Define the path to the 'wetland_categories' folder
folder_to_zip = '/content/Hawaii_2005_009'
zip_file_path = '/content/Hawaii_2005_009_subset.zip'
shutil.make_archive(zip_file_path[:-4], 'zip', folder_to_zip)
print(f'ZIP file created: {zip_file_path}')

ZIP file created: /content/Hawaii_2005_009_subset.zip


In [16]:
# Define the destination folder in Google Drive
drive_folder = '/content/drive/MyDrive/wetland_unet/UNET_Image_Chips/imagechip_trainingdata'
shutil.move(zip_file_path, drive_folder)
print(f'ZIP file moved to Google Drive: {drive_folder}')
shutil.move(csv_file_path, drive_folder)
print(f'CSV file moved to Google Drive: {drive_folder}')

ZIP file moved to Google Drive: /content/drive/MyDrive/wetland_unet/UNET_Image_Chips/imagechip_trainingdata
CSV file moved to Google Drive: /content/drive/MyDrive/wetland_unet/UNET_Image_Chips/imagechip_trainingdata
